# Assignment 2

### Imports

In [ ]:
import mlrose_hiive
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import time

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

### Problem Definitions

In [ ]:
# Problem Definitions
OMFitness = mlrose_hiive.OneMax()
OMProblemLength = 20
OMProblem = mlrose_hiive.DiscreteOpt(length=OMProblemLength, fitness_fn=OMFitness)
OMInit = np.random.choice([0, 1], size=(OMProblemLength,))

coords = [(0, 0), (4, 0), (1, 2), (3, 4), (5, 3), 
            (3, 2), (1, 5), (2, 1), (1, 6), (7, 2), 
            (3, 3), (0, 4), (2, 3), (3, 9), (3, 1),
            (1, 1), (2, 0), (5, 6), (7, 9), (7, 4)]
TSProblemLength = 10
coordsTrimmed = coords[:TSProblemLength]
TSFitness = mlrose_hiive.TravellingSales(coords=coordsTrimmed)
TSProblem = mlrose_hiive.TSPOpt(length=TSProblemLength, fitness_fn=TSFitness, maximize=False)
TSInit = np.arange(TSProblemLength)

FPFitness = mlrose_hiive.FourPeaks()
FPProblemLength = 20
FPProblem = mlrose_hiive.DiscreteOpt(length=FPProblemLength, fitness_fn=FPFitness)
FPInit = np.random.choice([0, 1], size=(FPProblemLength,))

fitnesses = [TSFitness, OMFitness, FPFitness]
problems = [TSProblem, OMProblem, FPProblem]
init_states = [TSInit, OMInit, FPInit]
problemNames = ["Travelling Salesman", "One Max", "Four Peaks"]

schedule = mlrose_hiive.GeomDecay()

# Fitness and Fevals by Iteration

In [ ]:
prob = 2 # TS, OM, FP
maxIters = 10000
maxAttempts = 200
popSize = 100

HC_state, HC_fitness, HC_curve = mlrose_hiive.random_hill_climb(problems[prob], max_attempts=maxAttempts, max_iters=maxIters, restarts=10, init_state=None, curve=True, random_state=1)
SA_state, SA_fitness, SA_curve = mlrose_hiive.simulated_annealing(problems[prob], schedule=schedule, max_attempts=maxAttempts, max_iters=maxIters, init_state=None, curve=True, random_state=1)
GA_state, GA_fitness, GA_curve = mlrose_hiive.genetic_alg(problems[prob], pop_size=popSize, mutation_prob=0.1, max_attempts=maxAttempts, max_iters=maxIters, curve=True, random_state=1)
MIMIC_state, MIMIC_fitness, MIMIC_curve = mlrose_hiive.mimic(problems[prob], pop_size=popSize, keep_pct=0.2, max_attempts=maxAttempts, max_iters=maxIters, curve=True, random_state=1)

In [ ]:
fitnesses = [HC_curve.T[0], SA_curve.T[0], GA_curve.T[0], MIMIC_curve.T[0]]
fevals = [HC_curve.T[1], SA_curve.T[1], GA_curve.T[1], MIMIC_curve.T[1]]

fig = plt.figure(1, [18, 6])
ax = fig.add_subplot(121)
ax.plot(np.arange(fitnesses[0].size), fitnesses[0])
ax.plot(np.arange(fitnesses[1].size), fitnesses[1])
ax.plot(np.arange(fitnesses[2].size), fitnesses[2])
ax.plot(np.arange(fitnesses[3].size), fitnesses[3])
ax.set_title("Fitness by Model using Fitness of " + problemNames[prob])
ax.set_xlabel("Iteration")
ax.set_ylabel("Fitness")
ax.legend(["Random Hill Climbing", "Simulated Annealing", "Genetic Algorithm", "MIMIC"])

ax2 = fig.add_subplot(122)
ax2.plot(np.arange(fevals[0].size), fevals[0])
ax2.plot(np.arange(fevals[1].size), fevals[1])
ax2.plot(np.arange(fevals[2].size), fevals[2])
ax2.plot(np.arange(fevals[3].size), fevals[3])
ax2.set_title("Function Evaluations by Model using Fitness of " + problemNames[prob])
ax2.set_xlabel("Iteration")
ax2.set_ylabel("Function Evaluations")
ax2.legend(["Random Hill Climbing", "Simulated Annealing", "Genetic Algorithm", "MIMIC"])

# Fitness and Runtime by Problem Size

In [ ]:
starts = [4, 4, 4] # TS, OM, FP
ends = [20, 40, 40] # TS, OM, FP

data = np.zeros((3, max(ends) - min(starts) + 1, 4)) # [problem][problemSize][algorithm]
times = np.zeros((3, max(ends) - min(starts) + 1, 4))

In [ ]:
prob2 = 1 # TS, OM, FP

In [ ]:
for j in np.arange(3):
    print(problemNames[j])
    for i in np.arange(starts[j], ends[j] + 1):
        print(i)
        if j == 0:
            coordsTrimmed2 = coords[:i]
            test2_fitness = mlrose_hiive.TravellingSales(coords=coordsTrimmed2)
            test2_problem = mlrose_hiive.TSPOpt(length=i, fitness_fn=test2_fitness, maximize=False)
            test2_init = np.arange(i)
        elif j == 1:
            test2_fitness = mlrose_hiive.OneMax()
            test2_problem = mlrose_hiive.DiscreteOpt(length=i, fitness_fn=test2_fitness)
            test2_init = np.random.choice([0, 1], size=(i,))
        elif j == 2:
            test2_fitness = mlrose_hiive.FourPeaks()
            test2_problem = mlrose_hiive.DiscreteOpt(length=i, fitness_fn=test2_fitness)
            test2_init = np.random.choice([0, 1], size=(i,))

        maxIters = 10000
        popSize = 200

        startTime = time.time()
        HC_state, HC_fitness, _ = mlrose_hiive.random_hill_climb(test2_problem, max_attempts=100, max_iters=maxIters, restarts=10, init_state=test2_init, random_state=1)
        t1 = time.time()
        SA_state, SA_fitness, _ = mlrose_hiive.simulated_annealing(test2_problem, schedule=schedule, max_attempts=100, max_iters=maxIters, init_state=test2_init, random_state=1)
        t2 = time.time()
        GA_state, GA_fitness, _ = mlrose_hiive.genetic_alg(test2_problem, pop_size=popSize, mutation_prob=0.1, max_attempts=100, max_iters=maxIters, random_state=1)
        t3 = time.time()
        MIMIC_state, MIMIC_fitness, _ = mlrose_hiive.mimic(test2_problem, pop_size=popSize, keep_pct=0.2, max_attempts=100, max_iters=maxIters, random_state=1)
        t4 = time.time()

        data[j][:][i - starts[prob2]] = [HC_fitness, SA_fitness, GA_fitness, MIMIC_fitness]
        times[j][:][i - starts[prob2]] = [t1 - startTime, t2 - t1, t3 - t2, t4 - t3]

In [ ]:
plotProb = 2 # TS, OM, FP

xax = np.arange(starts[plotProb], ends[plotProb] + 1)
sampleNum = ends[plotProb] - starts[plotProb] + 1

probData = data[plotProb][:sampleNum][:]
probTime = times[plotProb][:sampleNum][:]

fig = plt.figure(1, [12, 6])
ax = fig.add_subplot(111)
ax.plot(xax, probData.T[0])
ax.plot(xax, probData.T[1])
ax.plot(xax, probData.T[2])
ax.plot(xax, probData.T[3])
ax.set_title("Fitness by Problem Size using Fitness of " + problemNames[plotProb])
ax.set_xlabel("Problem Size")
ax.set_ylabel("Fitness")
ax.legend(["Hill Climbing", "Simulated Annealing", "Genetic Algorithm", "MIMIC"])

In [ ]:
plotProb = 2 # TS, OM, FP

xax = np.arange(starts[plotProb], ends[plotProb] + 1)
sampleNum = ends[plotProb] - starts[plotProb] + 1

probData = data[plotProb][:sampleNum][:]
probTime = times[plotProb][:sampleNum][:]

fig = plt.figure(1, [12, 6])
ax = fig.add_subplot(111)
ax.plot(xax, probTime.T[0])
ax.plot(xax, probTime.T[1])
ax.plot(xax, probTime.T[2])
ax.plot(xax, probTime.T[3])
ax.set_title("Runtime by Model using Fitness of " + problemNames[plotProb])
ax.set_xlabel("Problem Size")
ax.set_ylabel("Runtime")
ax.legend(["Random Hill Climbing", "Simulated Annealing", "Genetic Algorithm", "MIMIC"])

## Neural Network Fitting

In [ ]:
iris = load_iris()

irisData = iris.data.T[:2].T
irisTarget = iris.target

trainFeatures, testFeatures, trainTargets, testTargets = train_test_split(irisData, irisTarget, test_size = 0.2, random_state = 1)

one_hot = OneHotEncoder()
trainTargets = np.asarray(one_hot.fit_transform(trainTargets.reshape(-1, 1)).todense())
testTargets = np.asarray(one_hot.fit_transform(testTargets.reshape(-1, 1)).todense())

nn_model1 = mlrose_hiive.NeuralNetwork(hidden_nodes=[4], activation='relu', algorithm='random_hill_climb', 
                                max_iters=100000, bias=True, is_classifier=True, learning_rate=0.1, 
                                early_stopping=True, clip_max=5, max_attempts=100, random_state=None, curve=True, restarts=100)

nn_model1.fit(trainFeatures, trainTargets)

nn1trainPred = nn_model1.predict(trainFeatures)
trainAccuracy1 = accuracy_score(nn1trainPred, trainTargets)

nn1TestPred = nn_model1.predict(testFeatures)
testAccuracy1 = accuracy_score(nn1TestPred, testTargets)

print(trainAccuracy1, testAccuracy1)

In [ ]:
nn_model2 = mlrose_hiive.NeuralNetwork(hidden_nodes=[4], activation='relu', algorithm='simulated_annealing', 
                                max_iters=100000, bias=True, is_classifier=True, learning_rate=0.1, 
                                early_stopping=True, clip_max=5, max_attempts=100, random_state=1, curve=True)
nn_model2.fit(trainFeatures, trainTargets)

nn2trainPred = nn_model2.predict(trainFeatures)
trainAccuracy2 = accuracy_score(nn2trainPred, trainTargets)
nn2TestPred = nn_model2.predict(testFeatures)
testAccuracy2 = accuracy_score(nn2TestPred, testTargets)

print(trainAccuracy2, testAccuracy2)


In [ ]:
nn_model3 = mlrose_hiive.NeuralNetwork(hidden_nodes=[4], activation='relu', algorithm='genetic_alg', 
                                max_iters=100000, bias=True, is_classifier=True, learning_rate=0.1, 
                                early_stopping=True, clip_max=5, max_attempts=100, random_state=1, pop_size=200, curve=True)
nn_model3.fit(trainFeatures, trainTargets)

nn3trainPred = nn_model3.predict(trainFeatures)
trainAccuracy3 = accuracy_score(nn3trainPred, trainTargets)
nn3TestPred = nn_model3.predict(testFeatures)
testAccuracy3 = accuracy_score(nn3TestPred, testTargets)

print(trainAccuracy3, testAccuracy3)

In [ ]:
NNfitnesses = [nn_model1.fitness_curve.T[0], nn_model2.fitness_curve.T[0], nn_model3.fitness_curve.T[0]]
NNfevals = [nn_model1.fitness_curve.T[1], nn_model2.fitness_curve.T[1], nn_model3.fitness_curve.T[1]]

fig = plt.figure(1, [18, 8])
ax = fig.add_subplot(121)
ax.plot(NNfitnesses[0])
ax.plot(NNfitnesses[1])
ax.plot(NNfitnesses[2])

ax.set_title("Neural Net Weight Fitting Fitness over Iteration")
ax.set_xlabel("Iteration")
ax.set_ylabel("Loss")
ax.legend(["Random Hill Climbing", "Simulated Annealing", "Genetic Algorithm"])

ax = fig.add_subplot(122)
ax.plot(NNfevals[0])
ax.plot(NNfevals[1])
ax.plot(NNfevals[2])

ax.set_title("Fitness Evals over Iteration")
ax.set_xlabel("Iteration")
ax.set_ylabel("Function Evaluations")
ax.legend(["Random Hill Climbing", "Simulated Annealing", "Genetic Algorithm"])